In [16]:
%run "contant.py"

已加载全局配置
🤖 可用Agent打印方法: print_last_message(), print_agent(), agent_reply()


In [17]:
from langchain.agents import create_agent
from langgraph.checkpoint.memory import InMemorySaver  
from langchain_core.messages import HumanMessage

@before_model
def custom_message_filter(state, runtime: Runtime) -> dict[str, Any] | None:
    print_json(state)
    print_json(state)

# 创建检查点保存器
checkpointer = InMemorySaver()

# 创建带记忆的 Agent
agent = create_agent(
    OPENAI_MODEL,
    middleware=[custom_message_filter],
    checkpointer=checkpointer,  
)

# 使用 thread_id 区分不同对话
config1 = {"configurable": {"thread_id": "1"}}
config2 = {"configurable": {"thread_id": "2"}}

# 第一次调用 - Agent 会记住用户名
result1 = agent.invoke(
    {"messages": [HumanMessage("你好！我的名字是鲍勃。")]},
    config1
)

print(result1["messages"][-1].content) 

# 第二次调用 - Agent 还记得用户名
result2 = agent.invoke(
    {"messages": [HumanMessage("我的名字是什么？")]},
    config1
)
print(result2["messages"][-1].content) 


# 第三次调用 - 换了config2
result3 = agent.invoke(
    {"messages": [HumanMessage("我的名字是什么？")]},
    config2
)
print(result3["messages"][-1].content) 

{
  "messages": [
    {
      "content": "你好！我的名字是鲍勃。",
      "additional_kwargs": {},
      "response_metadata": {},
      "type": "human",
      "name": null,
      "id": "23c34942-3ddb-495f-9249-eaab435e6195"
    }
  ]
}
{
  "messages": [
    {
      "content": "你好！我的名字是鲍勃。",
      "additional_kwargs": {},
      "response_metadata": {},
      "type": "human",
      "name": null,
      "id": "23c34942-3ddb-495f-9249-eaab435e6195"
    }
  ]
}
你好，鲍勃！很高兴认识你。有什麼我可以帮你的吗？
{
  "messages": [
    {
      "content": "你好！我的名字是鲍勃。",
      "additional_kwargs": {},
      "response_metadata": {},
      "type": "human",
      "name": null,
      "id": "23c34942-3ddb-495f-9249-eaab435e6195"
    },
    {
      "content": "你好，鲍勃！很高兴认识你。有什麼我可以帮你的吗？",
      "additional_kwargs": {
        "refusal": null
      },
      "response_metadata": {
        "token_usage": {
          "completion_tokens": 22,
          "prompt_tokens": 17,
          "total_tokens": 39,
          "completion_tokens_details": {
    

### 修剪消息 (Trim Messages)

In [ ]:
from langchain.agents import create_agent
from langchain.agents.middleware import before_model
from langchain_core.messages import HumanMessage, RemoveMessage
from langgraph.graph.message import REMOVE_ALL_MESSAGES
from langgraph.checkpoint.memory import InMemorySaver
from langgraph.runtime import Runtime
from typing import Any

# 修剪消息中间件 - 只保留最后3条消息
@before_model
def trim_messages(state, runtime: Runtime) -> dict[str, Any] | None:
    """只保留最后3条消息以适应上下文窗口"""
    messages = state["messages"]
    
    # 如果消息不超过3条，无需修剪
    if len(messages) <= 3:
        return None
    
    # 保留第一条和最后3条
    first_msg = messages[0]
    recent_messages = messages[-3:]
    new_messages = [first_msg] + recent_messages
    
    print(f"🔄 修剪消息：从 {len(messages)} 条减少到 {len(new_messages)} 条")
    
    return {
        "messages": [
            RemoveMessage(id=REMOVE_ALL_MESSAGES),
            *new_messages
        ]
    }

# 创建带修剪功能的Agent
checkpointer = InMemorySaver()
agent = create_agent(
    OPENAI_MODEL,
    middleware=[trim_messages],
    checkpointer=checkpointer
)

config = {"configurable": {"thread_id": "trim_demo"}}

print("=== 修剪消息演示 ===")

# 模拟长对话
messages = [
    "你好，我是张三",
    "我喜欢编程",
    "今天天气不错",
    "我想学习Python",
    "请推荐一些学习资源",
    "谢谢你的建议"
]

for i, msg in enumerate(messages, 1):
    print(f"\n第{i}轮对话: {msg}")
    result = agent.invoke({"messages": [HumanMessage(msg)]}, config)
    print(f"Agent回复: {result['messages'][-1].content[:50]}...")

print("\n✅ 修剪消息演示完成 - Agent始终记住关键信息")

=== 修剪消息演示 ===

第1轮对话: 你好，我是张三
Agent回复: 你好，张三！很高兴见到你 😊 有什么可以帮助你的？...

第2轮对话: 我喜欢编程
Agent回复: 太棒了！编程是一项非常有趣又实用的技能，可以让你创造不同的事物并解决各种问题。你有特别喜欢的编程语言...

第3轮对话: 今天天气不错
🔄 修剪消息：从 5 条减少到 4 条
Agent回复: 是啊，天气好总能让人心情更愉悦！这样的日子特别适合出去走走，散散心，或者静下来做些自己喜欢的事情。你...

第4轮对话: 我想学习Python
🔄 修剪消息：从 6 条减少到 4 条
Agent回复: 太棒了！Python是一门非常受欢迎、易于入门且功能强大的编程语言，无论是数据分析、人工智能还是We...

第5轮对话: 请推荐一些学习资源
🔄 修剪消息：从 6 条减少到 4 条
Agent回复: 当然可以！以下是一些专门为了学习Python的高质量资源，从在线教程、学习平台到书籍和视频课程，我将...

第6轮对话: 谢谢你的建议
🔄 修剪消息：从 6 条减少到 4 条
Agent回复: 不客气！很高兴能够帮到你 😊  
如果在学习过程中遇到任何问题，比如代码逻辑、技术概念或者项目实战的...

✅ 修剪消息演示完成 - Agent始终记住关键信息


### 删除消息 (Delete Messages)

In [8]:
from langchain.agents import create_agent
from langchain.agents.middleware import after_model
from langchain_core.messages import HumanMessage, RemoveMessage
from langgraph.checkpoint.memory import InMemorySaver
from langgraph.runtime import Runtime
from typing import Any

# 删除消息中间件 - 删除包含敏感词的消息
@after_model
def delete_sensitive_messages(state, runtime: Runtime) -> dict | None:
    """删除包含敏感词的消息"""
    messages = state["messages"]
    
    # 敏感词列表
    SENSITIVE_WORDS = ["密码", "secret", "token"]
    
    for message in messages:
        if any(word in message.content for word in SENSITIVE_WORDS):
            print(f"🗑️ 删除敏感消息: {message.content[:30]}...")
            return {"messages": [RemoveMessage(id=message.id)]}
    
    return None

# 创建带删除功能的Agent
checkpointer = InMemorySaver()
agent = create_agent(
    OPENAI_MODEL,
    middleware=[delete_sensitive_messages],
    checkpointer=checkpointer
)

config = {"configurable": {"thread_id": "delete_demo"}}

print("=== 删除消息演示 ===")

# 测试消息
test_messages = [
    "你好，我想咨询一些问题",
    "我的密码是123456，请帮我保存",  # 包含敏感词
    "我的密码是什么"
]

for msg in test_messages:
    print(f"\n用户: {msg}")
    result = agent.invoke({"messages": [HumanMessage(msg)]}, config)
    print_agent(result)

print("\n✅ 删除消息演示完成 - 敏感信息被自动清理")

=== 删除消息演示 ===

用户: 你好，我想咨询一些问题
Agent: 您好！请问有什么问题我可以帮您解答的？

用户: 我的密码是123456，请帮我保存
🗑️ 删除敏感消息: 我的密码是123456，请帮我保存...
Agent: 您好！为了您的账户安全，建议您不要将密码直接在公共平台或不安全的地方保存。如果您需要帮助设置或管理密码，可以考虑使用密码管理工具，或者使用复杂且唯一的密码以保护您的账户安全。如果有其他问题，欢迎随时告诉我！

用户: 我的密码是什么
🗑️ 删除敏感消息: 您好！为了您的账户安全，建议您不要将密码直接在公共平台或不安...
Agent: 抱歉，我无法知道您的密码。如果您忘记了密码，可以尝试使用“忘记密码”功能进行重置，或者联系相关平台的客服获取帮助。有什么我可以帮您指导的地方吗？

✅ 删除消息演示完成 - 敏感信息被自动清理


### 案例3：总结消息 (Summarize Messages)

In [ ]:
from langchain.agents import create_agent
from langchain.agents.middleware import SummarizationMiddleware
from langchain_core.messages import HumanMessage
from langgraph.checkpoint.memory import InMemorySaver
from langchain_core.runnables import RunnableConfig

# 创建总结中间件
summarization_middleware = SummarizationMiddleware(
    OPENAI_MODEL,
    trigger=("tokens", 1000),  # 当token超过1000时触发总结
    keep=("messages", 5)       # 保留最近5条消息
)

# 创建带总结功能的Agent
checkpointer = InMemorySaver()
agent = create_agent(
    OPENAI_MODEL,
    middleware=[summarization_middleware],
    checkpointer=checkpointer
)

config: RunnableConfig = {"configurable": {"thread_id": "summary_demo"}}

print("=== 总结消息演示 ===")

# 模拟长对话
long_conversation = [
    "你好，我是李四，是一名软件工程师",
    "我正在开发一个电商网站",
    "网站需要用户认证功能",
    "还要有商品展示和购物车",
    "支付接口也很重要",
    "我考虑使用React作为前端框架"
]

for i, msg in enumerate(long_conversation, 1):
    print(f"\n第{i}条消息: {msg}")
    result = agent.invoke({"messages": [HumanMessage(msg)]}, config)
    
    # 显示当前消息数量
    msg_count = len(result["messages"])
    print(f"当前消息数量: {msg_count}")

# 测试记忆效果
print("\n🧪 测试记忆效果:")
result = agent.invoke(
    {"messages": [HumanMessage("请总结一下我的项目需求")]}, 
    config
)
print(f"Agent回复: {result['messages'][-1].content}")

print("\n✅ 总结消息演示完成 - 长对话被智能压缩")

=== 总结消息演示 ===

第1条消息: 你好，我是李四，是一名软件工程师
当前消息数量: 2

第2条消息: 我正在开发一个电商网站
当前消息数量: 4

第3条消息: 网站需要用户认证功能
当前消息数量: 6

第4条消息: 还要有商品展示和购物车
当前消息数量: 8

第5条消息: 支付接口也很重要
当前消息数量: 10

第6条消息: 我考虑使用React作为前端框架
当前消息数量: 12

🧪 测试记忆效果:
Agent回复: 当然可以，以下是对你项目需求的总结：

---

**项目名称：** 电商网站开发

**技术栈：** 前端使用React

**主要功能需求：**

1. **用户认证**
   - 注册和登录功能
   - 支持密码重置、邮箱验证（可选）
   - 使用JWT或类似方案管理用户会话

2. **商品展示**
   - 商品列表页面：支持分页、筛选（类别、价格等）、排序
   - 商品详情页面：展示详细信息（图片、描述、价格、库存、评价）

3. **购物车**
   - 添加商品到购物车
   - 支持修改商品数量和删除商品
   - 展示购物车内容及总价
   - 流程引导用户结算

4. **订单结算与支付**
   - 订单确认页面
   - 支付接口集成（支持微信支付、支付宝等）
   - 支付成功后更新订单状态，完成交易流程
   - 支付异步通知处理（后端实现）

5. **后台管理（可扩展）**
   - 商品管理
   - 订单管理
   - 用户管理（可选）

**附加考虑：**
- 页面UI设计建议使用UI组件库（如Ant Design）
- 前端状态管理用Redux或Context API
- 后端API（虽然未明确提及具体后端技术，但需要支持上述前端功能）
- 确保数据交互安全（使用HTTPS、Token管理等）

---

如果你需要我帮助完善方案、设计架构或提供具体实现示例，随时告诉我！

✅ 总结消息演示完成 - 长对话被智能压缩


### 案例4：自定义策略 (Custom Strategy)

In [13]:
from langchain.agents import create_agent
from langchain.agents.middleware import before_model
from langchain_core.messages import HumanMessage, RemoveMessage, AIMessage
from langgraph.checkpoint.memory import InMemorySaver
from langgraph.runtime import Runtime
from typing import Any
import re

# 自定义消息过滤中间件
@before_model
def custom_message_filter(state, runtime: Runtime) -> dict[str, Any] | None:
    """自定义消息过滤策略"""
    messages = state["messages"]

    jprint(messages)

    # messages_to_remove = []
    
    # for message in messages:
    #     # 策略1: 删除过短的消息（少于5个字符）
    #     if len(message.content.strip()) < 5:
    #         messages_to_remove.append(message.id)
    #         print(f"🚫 删除过短消息: '{message.content}'")
    #         continue
        
    #     # 策略2: 删除重复的问候语
    #     if message.content.lower() in ["你好", "hello", "hi"]:
    #         # 检查是否已有类似问候
    #         for other_msg in messages:
    #             if (other_msg.id != message.id and 
    #                 other_msg.content.lower() in ["你好", "hello", "hi"]):
    #                 messages_to_remove.append(message.id)
    #                 print(f"🚫 删除重复问候: '{message.content}'")
    #                 break
        
    #     # 策略3: 删除纯数字消息（可能是误输入）
    #     if message.content.isdigit():
    #         messages_to_remove.append(message.id)
    #         print(f"🚫 删除纯数字消息: '{message.content}'")
    
    # if messages_to_remove:
    #     return {"messages": [RemoveMessage(id=msg_id) for msg_id in messages_to_remove]}
    
    # return None

# 创建带自定义策略的Agent
checkpointer = InMemorySaver()
agent = create_agent(
    OPENAI_MODEL,
    middleware=[custom_message_filter],
    checkpointer=checkpointer
)

config = {"configurable": {"thread_id": "custom_demo"}}

print("=== 自定义策略演示 ===")

# 测试各种消息类型
test_messages = [
    "你好",           # 重复问候
    "我叫小明",          # 重复问候
    # "123",            # 纯数字
    # "hi",             # 重复问候
    # "好的",           # 正常消息
    # "嗯",             # 过短消息
    # "我想学习编程",    # 正常消息
    # "456",            # 纯数字
    # "请推荐一些资源"   # 正常消息
]

for msg in test_messages:
    print(f"\n输入: '{msg}'")
    result = agent.invoke({"messages": [HumanMessage(msg)]}, config)
    
    # 显示当前消息数量
    current_count = len([m for m in result["messages"] if m.type != "system"])
    print(f"有效消息数量: {current_count}")

print("\n✅ 自定义策略演示完成 - 消息被智能过滤")

=== 自定义策略演示 ===

输入: '你好'
[
  {
    "content": "你好",
    "additional_kwargs": {},
    "response_metadata": {},
    "type": "human",
    "name": null,
    "id": "e93b4dd5-04a5-4518-aed9-f3e313f09d52"
  }
]
有效消息数量: 2

输入: '我叫小明'
[
  {
    "content": "你好",
    "additional_kwargs": {},
    "response_metadata": {},
    "type": "human",
    "name": null,
    "id": "e93b4dd5-04a5-4518-aed9-f3e313f09d52"
  },
  {
    "content": "你好！很高兴为你服务。有什么我可以帮忙的吗？",
    "additional_kwargs": {
      "refusal": null
    },
    "response_metadata": {
      "token_usage": {
        "completion_tokens": 17,
        "prompt_tokens": 8,
        "total_tokens": 25,
        "completion_tokens_details": {
          "accepted_prediction_tokens": 0,
          "audio_tokens": 0,
          "reasoning_tokens": 0,
          "rejected_prediction_tokens": 0
        },
        "prompt_tokens_details": {
          "audio_tokens": 0,
          "cached_tokens": 0,
          "cache_creation_input_tokens": 0,
          "cache_rea